### Data Download Notebook

* Please don't run this notebook. This notebook is meant for downloading data (which takes around 1hr) and saving the data in a pickle format to be used later anytime. I have also shared few insights I got from data while exploring
 
* Please refer submission.ipynb for code related to features and backtesting

In [14]:
import pandas as pd 
import numpy as np
import baostock as bs
from tqdm import tqdm
import matplotlib.pyplot as plt 

In [ ]:
#Uncomment below if baostock is not installed

#  !pip install baostock

### Download the index composition of CSI500 index at date = '2021-01-01'. 

In [8]:
lg = bs.login()
print('login respond error_code:'+lg.error_code)
print('login respond error_msg:'+lg.error_msg)

login success!
login respond error_code:0
login respond error_msg:success


In [9]:
rs = bs.query_zz500_stocks(date = '2021-01-01')
print('query_zz500 error_code:'+rs.error_code)
print('query_zz500 error_msg:'+rs.error_msg)

query_zz500 error_code:0
query_zz500 error_msg:success


In [10]:

zz500_stocks = []
while (rs.error_code == '0') & rs.next():
    # Get a record and merge the records together
    zz500_stocks.append(rs.get_row_data())
result = pd.DataFrame(zz500_stocks, columns=rs.fields)

result.to_csv("./zz500_stocks.csv", encoding="gbk", index=False)
print(result)



     updateDate       code code_name
0    2020-12-28  sh.600006      东风汽车
1    2020-12-28  sh.600008      首创股份
2    2020-12-28  sh.600021      上海电力
3    2020-12-28  sh.600022      山东钢铁
4    2020-12-28  sh.600026      中远海能
..          ...        ...       ...
495  2020-12-28  sz.300496      中科创达
496  2020-12-28  sz.300595      欧普康视
497  2020-12-28  sz.300618      寒锐钴业
498  2020-12-28  sz.300630      普利制药
499  2020-12-28  sz.300699      光威复材

[500 rows x 3 columns]


maybe I can use the code column as unique identifier

In [11]:
bs.logout()

logout success!


###  Download 30min bar data from 2022-04-01 to 2022-07-31 for all 500 stocks of the CSI500 index


In [19]:


bs.login()


start_date = '2022-04-01'
end_date = '2022-07-31'
frequency = '30'
market_data = []
writer = pd.ExcelWriter("market_data.xlsx", engine="xlsxwriter")
# Iteratre through each stock
for stock in tqdm(result['code'], desc= 'Downloading Data'): 
    
    rs = bs.query_history_k_data_plus(stock, 
                                      "date,time,code,open,high,low,close,volume,amount,adjustflag", 
                                      start_date=start_date, 
                                      end_date=end_date, 
                                      frequency=frequency,
                                      adjustflag='3')

    # Check if the query was successful
    if rs.error_code == '0':
        data = rs.get_data()
        market_data.append(data)
        data.to_excel(writer, sheet_name=stock)
    else:
        print(f"Failed to fetch data for {stock}. Error code: {rs.error_code}, Error message: {rs.error_msg}")
    
    

bs.logout()


login success!


logout success!


I saved it as excel because I didnot want to download the data again and again. It takes 1 hr to dowload. While it was dowloading I remembered about pickle. Which is even better method than excel file. So doing that in code below

At this point all required data is downloaded

In [22]:
#  pickle module is inbuilt 
import pickle

with open("market_data.pkl", "wb") as f:
  pickle.dump(market_data, f)



#### Data Cleaning 

In [62]:
market_data[60]

,date,time,code,open,high,low,close,volume,amount,adjustflag
0,2022-04-01,20220401100000000,sh.600392,16.9000,17.1500,16.8200,17.1100,5547792,94542824.0000,3
1,2022-04-01,20220401103000000,sh.600392,17.1100,17.2500,17.0900,17.2300,4363700,74899088.0000,3
2,2022-04-01,20220401110000000,sh.600392,17.2300,17.3300,17.1600,17.2300,3904800,67352245.0000,3
3,2022-04-01,20220401113000000,sh.600392,17.2300,17.2600,17.1200,17.1300,1830900,31482266.0000,3
4,2022-04-01,20220401133000000,sh.600392,17.1300,17.2000,17.1200,17.1500,1328300,22791065.0000,3
...,...,...,...,...,...,...,...,...,...,...
635,2022-07-29,20220729113000000,sh.600392,20.9200,21.0500,20.8000,20.8100,1708800,35693046.0000,3
636,2022-07-29,20220729133000000,sh.600392,20.8100,20.8600,20.7100,20.8300,2002769,41604480.0000,3
637,2022-07-29,20220729140000000,sh.600392,20.8200,20.9000,20.7500,20.8300,1905300,39668896.0000,3
638,2022-07-29,20220729143000000,sh.600392,20.8500,20.8500,20.7100,20.7700,1702194,35339968.0000,3


In [32]:
df = pd.concat(market_data)


In [33]:
df

,date,time,code,open,high,low,close,volume,amount,adjustflag
0,2022-04-01,20220401100000000,sh.600006,5.4500,5.4500,5.4000,5.4300,2019505,10960302.0000,3
1,2022-04-01,20220401103000000,sh.600006,5.4300,5.4700,5.4300,5.4600,983100,5359361.0000,3
2,2022-04-01,20220401110000000,sh.600006,5.4600,5.4800,5.4300,5.4600,854700,4667252.0000,3
3,2022-04-01,20220401113000000,sh.600006,5.4600,5.4700,5.4400,5.4400,712300,3887314.0000,3
4,2022-04-01,20220401133000000,sh.600006,5.4400,5.4500,5.4300,5.4300,481100,2616551.0000,3
...,...,...,...,...,...,...,...,...,...,...
635,2022-07-29,20220729113000000,sz.300699,70.3400,70.6900,70.0000,70.0700,553807,38977724.0000,3
636,2022-07-29,20220729133000000,sz.300699,70.0300,70.0300,69.3000,69.6000,432687,30084125.0000,3
637,2022-07-29,20220729140000000,sz.300699,69.7000,69.9600,69.6000,69.7500,243000,16955343.0000,3
638,2022-07-29,20220729143000000,sz.300699,69.7300,70.8800,69.6600,70.8800,751600,52897905.0000,3


it has 319008 rows. I noticed most stocks have 640 rows and we have 500 stocks so total rows cannot end with a '8'

In [88]:
saveI = []
save_code = []
#for stocks with missing data save the code(its like a stock_id) and its index in market_data list
for i in range(len(market_data)):
    if market_data[i].shape[0] != 640:
        saveI.append(i)
        save_code.append(market_data[i]['code'][0])


In [94]:
f = 0
for i in saveI:
    print(save_code[f],market_data[i].shape)
    f = f+1

sh.600006 (608, 10)
sh.600053 (624, 10)
sh.600260 (632, 10)
sh.600291 (272, 10)
sh.600372 (560, 10)
sh.600388 (600, 10)
sh.600515 (632, 10)
sh.600804 (632, 10)
sz.000564 (632, 10)
sz.000732 (632, 10)
sz.000830 (560, 10)
sz.002013 (560, 10)
sz.002670 (400, 10)
sz.300070 (624, 10)


interesting we have few stocks for which complete data is not present


In [46]:
saveI

[0, 8, 39, 44, 55, 59, 76, 128, 285, 301, 311, 337, 429, 464]

In [47]:
640*500 - 319008

992

let me print one with lowest value to confirm myself

In [58]:
market_data[44]

,date,time,code,open,high,low,close,volume,amount,adjustflag
0,2022-04-01,20220401100000000,sh.600291,3.9700,3.9700,3.8100,3.8100,16286500,62446944.0000,3
1,2022-04-01,20220401103000000,sh.600291,3.8100,3.8400,3.8100,3.8100,4719900,18001171.0000,3
2,2022-04-01,20220401110000000,sh.600291,3.8100,3.8100,3.8100,3.8100,653700,2490597.0000,3
3,2022-04-01,20220401113000000,sh.600291,3.8100,3.8100,3.8100,3.8100,506500,1929765.0000,3
4,2022-04-01,20220401133000000,sh.600291,3.8100,3.8100,3.8100,3.8100,351700,1339977.0000,3
...,...,...,...,...,...,...,...,...,...,...
267,2022-06-07,20220607113000000,sh.600291,0.9200,0.9400,0.9200,0.9400,1012961,943778.0000,3
268,2022-06-07,20220607133000000,sh.600291,0.9300,0.9500,0.9300,0.9400,3469356,3260566.0000,3
269,2022-06-07,20220607140000000,sh.600291,0.9400,0.9500,0.9300,0.9400,2913913,2749220.0000,3
270,2022-06-07,20220607143000000,sh.600291,0.9400,0.9400,0.9000,0.9100,4998731,4616127.0000,3


could be issue with download, Ill try dowloading this stock again - which has only 272 rows

In [63]:
bs.login()
rs = bs.query_history_k_data_plus('sh.600291', 
                                      "date,time,code,open,high,low,close,volume,amount,adjustflag", 
                                      start_date='2022-04-01', 
                                      end_date='2022-07-31', 
                                      frequency='30',
                                      adjustflag='3')
bs.logout()
temp = rs.get_data()
temp

login success!
logout success!


,date,time,code,open,high,low,close,volume,amount,adjustflag
0,2022-04-01,20220401100000000,sh.600291,3.9700,3.9700,3.8100,3.8100,16286500,62446944.0000,3
1,2022-04-01,20220401103000000,sh.600291,3.8100,3.8400,3.8100,3.8100,4719900,18001171.0000,3
2,2022-04-01,20220401110000000,sh.600291,3.8100,3.8100,3.8100,3.8100,653700,2490597.0000,3
3,2022-04-01,20220401113000000,sh.600291,3.8100,3.8100,3.8100,3.8100,506500,1929765.0000,3
4,2022-04-01,20220401133000000,sh.600291,3.8100,3.8100,3.8100,3.8100,351700,1339977.0000,3
...,...,...,...,...,...,...,...,...,...,...
267,2022-06-07,20220607113000000,sh.600291,0.9200,0.9400,0.9200,0.9400,1012961,943778.0000,3
268,2022-06-07,20220607133000000,sh.600291,0.9300,0.9500,0.9300,0.9400,3469356,3260566.0000,3
269,2022-06-07,20220607140000000,sh.600291,0.9400,0.9500,0.9300,0.9400,2913913,2749220.0000,3
270,2022-06-07,20220607143000000,sh.600291,0.9400,0.9400,0.9000,0.9100,4998731,4616127.0000,3


Looks like data is missing from their end. Now this is a problem. Can I really include these in my strategy ? what if I get buy signal and miss th sell signal ? - I am not sure why the data is missing so I cant decide yet

Let me see what data looks like in terms of values per day and total number of trading days

In [51]:
len(pd.unique(market_data[1]['date']))

80

we have 80 trading days so 640/80 = 8. for eaching trading day we have 8 values. I checked manually from 10 am to 3 pm it should be 11. Now what seems to be the problem lets check

In [56]:
pd.unique(market_data[1].loc[market_data[1]['date']=='2022-04-01']['time'])

array(['20220401100000000', '20220401103000000', '20220401110000000',
       '20220401113000000', '20220401133000000', '20220401140000000',
       '20220401143000000', '20220401150000000'], dtype=object)

so the data from 12, 1230pm and 1pm is not there. Is this the case with other stocks also that have 640 rows

In [61]:
for i in range(10,20):
    print(pd.unique(market_data[i].loc[market_data[i]['date']=='2022-04-01']['time']))

['20220401100000000' '20220401103000000' '20220401110000000'
 '20220401113000000' '20220401133000000' '20220401140000000'
 '20220401143000000' '20220401150000000']
['20220401100000000' '20220401103000000' '20220401110000000'
 '20220401113000000' '20220401133000000' '20220401140000000'
 '20220401143000000' '20220401150000000']
['20220401100000000' '20220401103000000' '20220401110000000'
 '20220401113000000' '20220401133000000' '20220401140000000'
 '20220401143000000' '20220401150000000']
['20220401100000000' '20220401103000000' '20220401110000000'
 '20220401113000000' '20220401133000000' '20220401140000000'
 '20220401143000000' '20220401150000000']
['20220401100000000' '20220401103000000' '20220401110000000'
 '20220401113000000' '20220401133000000' '20220401140000000'
 '20220401143000000' '20220401150000000']
['20220401100000000' '20220401103000000' '20220401110000000'
 '20220401113000000' '20220401133000000' '20220401140000000'
 '20220401143000000' '20220401150000000']
['20220401100000

looks like for all stocks data for 12/1230/1330 is not there. I checked online if the chinese market takes any break and turns out yes indeed they take break. so now I am only concerned for those stocks which have less than 640 rows. Stocks with 640 rows have complete data

In [67]:
for i in saveI:
    print(market_data[i].shape,80 - int(market_data[i].shape[0]/8)  )

(608, 10) 4
(624, 10) 2
(632, 10) 1
(272, 10) 46
(560, 10) 10
(600, 10) 5
(632, 10) 1
(632, 10) 1
(632, 10) 1
(632, 10) 1
(560, 10) 10
(560, 10) 10
(400, 10) 30
(624, 10) 2


in favor of time I am not digging deep into cause of why the data is missing. I will remove all stocks for which <90% is present. before I do that let me quickly ensure that there is no null in my data. 

In [86]:
df.isna().sum()

date          0
time          0
code          0
open          0
high          0
low           0
close         0
volume        0
amount        0
adjustflag    0
dtype: int64

In [103]:
df

,date,time,code,open,high,low,close,volume,amount,adjustflag
0,2022-04-01,20220401100000000,sh.600006,5.4500,5.4500,5.4000,5.4300,2019505,10960302.0000,3
1,2022-04-01,20220401103000000,sh.600006,5.4300,5.4700,5.4300,5.4600,983100,5359361.0000,3
2,2022-04-01,20220401110000000,sh.600006,5.4600,5.4800,5.4300,5.4600,854700,4667252.0000,3
3,2022-04-01,20220401113000000,sh.600006,5.4600,5.4700,5.4400,5.4400,712300,3887314.0000,3
4,2022-04-01,20220401133000000,sh.600006,5.4400,5.4500,5.4300,5.4300,481100,2616551.0000,3
...,...,...,...,...,...,...,...,...,...,...
635,2022-07-29,20220729113000000,sz.300699,70.3400,70.6900,70.0000,70.0700,553807,38977724.0000,3
636,2022-07-29,20220729133000000,sz.300699,70.0300,70.0300,69.3000,69.6000,432687,30084125.0000,3
637,2022-07-29,20220729140000000,sz.300699,69.7000,69.9600,69.6000,69.7500,243000,16955343.0000,3
638,2022-07-29,20220729143000000,sz.300699,69.7300,70.8800,69.6600,70.8800,751600,52897905.0000,3


In [141]:
remove_stocks  = (df['code'].value_counts() < 0.9*640)[(df['code'].value_counts() < 0.9*640)].index.tolist()
data_final  = df[df['code'].isin(remove_stocks)==False]
data_final

,date,time,code,open,high,low,close,volume,amount,adjustflag
0,2022-04-01,20220401100000000,sh.600006,5.4500,5.4500,5.4000,5.4300,2019505,10960302.0000,3
1,2022-04-01,20220401103000000,sh.600006,5.4300,5.4700,5.4300,5.4600,983100,5359361.0000,3
2,2022-04-01,20220401110000000,sh.600006,5.4600,5.4800,5.4300,5.4600,854700,4667252.0000,3
3,2022-04-01,20220401113000000,sh.600006,5.4600,5.4700,5.4400,5.4400,712300,3887314.0000,3
4,2022-04-01,20220401133000000,sh.600006,5.4400,5.4500,5.4300,5.4300,481100,2616551.0000,3
...,...,...,...,...,...,...,...,...,...,...
635,2022-07-29,20220729113000000,sz.300699,70.3400,70.6900,70.0000,70.0700,553807,38977724.0000,3
636,2022-07-29,20220729133000000,sz.300699,70.0300,70.0300,69.3000,69.6000,432687,30084125.0000,3
637,2022-07-29,20220729140000000,sz.300699,69.7000,69.9600,69.6000,69.7500,243000,16955343.0000,3
638,2022-07-29,20220729143000000,sz.300699,69.7300,70.8800,69.6600,70.8800,751600,52897905.0000,3


In [147]:
data_final['high']  = data_final.high.astype(float)
data_final['low'] = data_final.low.astype(float)
data_final['close'] = data_final.close.astype(float)
data_final['open'] = data_final.open.astype(float)
data_final['volume'] = data_final.volume.astype(float)
data_final.dtypes

C:\Users\akarte\AppData\Local\Temp\ipykernel_14296\201304878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final['high']  = data_final.high.astype(float)
C:\Users\akarte\AppData\Local\Temp\ipykernel_14296\201304878.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final['low'] = data_final.low.astype(float)
C:\Users\akarte\AppData\Local\Temp\ipykernel_14296\201304878.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

date           object
time           object
code           object
open          float64
high          float64
low           float64
close         float64
volume        float64
amount         object
adjustflag     object
dtype: object

In [148]:
#save the data in pickle format so that it can be used in another notebook. And I don't have to download/clean data everytime. 
# This file will be available in my github repo
with open("data_final.pkl", "wb") as f:
  pickle.dump(data_final, f)